# Scikit-learn: dane tekstowe

Aby przeprowadzić machine learning na dokumentach tekstowych, najpierw musimy zmienić treść tekstu na numeryczne wektory cech.

Reprezentacja bag-of-words
Przypisać stałą liczbę całkowitą id do każdego słowa występującego w dowolnym dokumencie zestawu treningowego 
(na przykład przez utworzenie słownika ze słów na indeksy). 
Dla każdego dokumentu j, zliczyć w nim liczbę wystąpień każdego słowa k, zapisując ją w X[k,j]

### Tokenizacja i zamiana tekstów na wektory

Pierwszy krok w analizie kolekcji dokumentów to tokenizacja i filtrowanie stop-wordów (ocjonalnie). Zawarte są w komponencie typu Vectorizer (np CountVectorizer), który buduje słownik mapowania słów na indeksy i przekształca dokumenty w wektory frekwencyjne bag-of-words.

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(['dokument pierwszy.', 'To jest tekst drugiego dokumentu'])

Zapoznaj się z innymi sposobami wektoryzacji dostępnymi w scikit-learn:
http://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text

### Trenowanie modelu klasyfikatora i predykcja

Dysponując cechami, funkcjami, możemy zacząć trenować klasyfikator, aby spróbować przewidzieć etykietę każdego dokumentu. Przykład dla klasyfikatora naiwnego Bayesa. Scikit-learn zawiera kilka wariantów tego klasyfikatora; Najbardziej odpowiedni do liczenia słów jest wariant MultinomialNB:

In [2]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, Y_train)

NameError: name 'X_train' is not defined

Wywołanie metody fit() powoduje dopasowanie się modelu do danych treningowych oraz etykiet tych danych.
Obiekt clf jest wytrenowanym modelem klasyfikatora który możemy używać do etykietowania nowych, niewidzianych obserwacji. Robimy to wywołując na nim funkcję predict:

In [ ]:
docs_new = ['No hipsters write tweets', 'Machine learning is fun?']
X_new_counts = count_vect.transform(docs_new)
predicted = clf.predict(X_new_tfidf)

### Tworzenie pipelines (potoków?)

Czasem warto dołożyć jeszcze kolejny krok przetwarzania, pomiędzy wektoryzacją a klasyfikatorem. Na przykład transformację TF-IDF, selekcję cech lub redukcję wymiarowości.

Aby uczynić sekwencję transformacji: wektorizer => transformator => klasyfikator łatwiejszą do pracy, scikit-learn udostępnia klasę Pipeline, która zachowuje się jak klasyfikator, jest jednak złożona z większej liczby elementów:

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

Na obiekcie tego typu działają metody fit i predict, typowe dla prostych klasyfikatorów.

In [6]:
_ = text_clf.fit(X_train, Y_train) # trenowanie modelu
Y_pred = text_clf.predict(X_test) # przewidywanie etykiet danych testowych (Y_pred)

### Ewaluacja modeli

Scikit-learn zapewnia narzędzia do analizy wyników:

In [7]:
from sklearn import metrics
print(metrics.classification_report(Y_true, Y_pred))  # szczegolowa analiza, wszystkie miary
print(metrics.confusion_matrix(Y_true, Y_pred))   # macierz konfuzji / confusion matrix

NameError: name 'Y_true' is not defined

wczytujemy plik tweets.csv (twitter airline sentiment z kaggle)
można użyć pandas, pd.read_csv(...)
   
Zakładamy, że kolumna 'airline_sentiment' zawiera 3 etykietowy sentyment, a 'text' zawartość tekstową tweeta.

Usuwamy ze zbioru wszystkie neutralne tweety ('airline_sentiment' = 0)

Dzielimy podziel zbiór  na treningowy, testowy i walidacyjny:
czyli: Y_train, Y_test, X_train, X_test, X_validate, Y_validate

In [11]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# ladowanie pliku z tweetami
df = pd.read_csv('Tweets.csv')

# usuniecie neutralnych tweetow
df = df[df['airline_sentiment'] != 'neutral']


len(df)
X = df['text']
Y = df['airline_sentiment']

X_train = X[:7000]
X_validate = X[7000:9000]
X_test = X[9000:]


Y_train = Y[:7000]
Y_validate = Y[7000:9000]
Y_test = Y[9000:]


#podmiana negative i positive na 0/1:
Y = Y.map({'negative': 0, 'positive': 1}).values

Ponizsza metoda, wywołana dla wytrenowanego pipeline'u, wypisze wagi najważniejszych parametrów

In [12]:
def print_significant_features(pipeline=None, n=20):
    feature_names = pipeline.get_params()['vect'].get_feature_names()
    coefs = []
    try:
        coefs = pipeline.get_params()['clf'].coef_
    except:
        coefs.append(pipeline.get_params()['clf'].feature_importances_)
    print("Total features: {}".format(len(coefs[0])))
    coefs_with_fns = sorted(zip(coefs[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_2, fn_2) in top:
        print( coef_2, str(fn_2) )

print_significant_features(text_clf)

Total features: 9663
(-9.640899778378873, '00') (-4.92304138598276, 'thanks')
(-9.640899778378873, '000419') (-4.949322391333496, 'you')
(-9.640899778378873, '000ft') (-4.974548270612971, 'thank')
(-9.640899778378873, '0016') (-5.0383728080318555, 'jetblue')
(-9.640899778378873, '00pm') (-5.1965545479339195, 'southwestair')
(-9.640899778378873, '01') (-5.330011179763391, 'the')
(-9.640899778378873, '0162389030167') (-5.485780904488961, 'united')
(-9.640899778378873, '0162424965446') (-5.503022883588678, 'for')
(-9.640899778378873, '0162431184663') (-5.514998288772777, 'to')
(-9.640899778378873, '01pm') (-5.812499697591749, 'great')
(-9.640899778378873, '02') (-5.982155740796642, 'and')
(-9.640899778378873, '04') (-6.012125273212083, 'virginamerica')
(-9.640899778378873, '05') (-6.018546209806735, 'it')
(-9.640899778378873, '0510') (-6.069229797205101, 'co')
(-9.640899778378873, '05am') (-6.096924085840985, 'flight')
(-9.640899778378873, '05pm') (-6.115699073224272, 'my')
(-9.6408997783

### Zadania

1) Stwórz pipeline wektoryzujący dane (możesz wypróbować również TF-IDF i różne dostępne sposoby!)
http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

2) do pipeline dodaj jeden z klasyfikatorów - wybór należy do Ciebie!
http://scikit-learn.org/stable/supervised_learning.html

3) wytrenuj model na danych treningowych (X_train i Y_train) - użyj metody fit(..) wywołanej na pipeline

4)  dokonaj jego ewaluacji na danych testowych (X_test i Y_test). Policz wyniki ewaluacji jako precyzję, recall, F1.

5) Wypróbuj modele klasyfikacyjne: Naive Bayes, MaxEnt, SVM, RandomForest. Zapisz wyniki ewaluacji na zbiorze testowym.

6) Spróbuj podnieść F1 na zbiorze testowym poprzez:
   - tuning przestrzeni hiperparametrów wybranych klasyfikatorów (nie używając w tym celu zbioru testowego!)
   - eksperymenty z różnymi obiektami Vectorizer i różnymi parametrami
   - normalizację przestrzeni cech (sklearn.preprocessing normalizer i normalize) dodaną do pieline

Wyniki udostępnij prowadzącemu o 16:00. Osoba o najwyższym F1 dostaje po przesłaniu swojego kodu, dodatkowy bonus punktowy do egzaminu zaliczającego przedmiot :)

In [69]:
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', LinearSVC()),
 ])

text_clf.fit(X_train, Y_train) # trenowanie modelu
Y_pred = text_clf.predict(X_test) # przewidywanie etykiet danych testowych (Y_pred)


print(metrics.classification_report(Y_test, Y_pred, digits=3))  # szczegolowa analiza, wszystkie miary



              precision    recall  f1-score   support

    negative      0.956     0.968     0.962      2163
    positive      0.801     0.743     0.771       378

   micro avg      0.934     0.934     0.934      2541
   macro avg      0.878     0.856     0.866      2541
weighted avg      0.933     0.934     0.933      2541



In [115]:
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.preprocessing import Binarizer
text_clf = Pipeline([('binarizer', Binarizer(copy=True, threshold=0.0)),
    #('vect', CountVectorizer()),
                     
                      ('tfidf', TfidfTransformer()),
                      ('clf', LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=False,
     intercept_scaling=1, max_iter=4000,
     multi_class='crammer_singer', penalty='l2', random_state=1, tol=1e-10)),
 ])

text_clf.fit(X_train, Y_train) # trenowanie modelu
Y_pred = text_clf.predict(X_test) # przewidywanie etykiet danych testowych (Y_pred)


print(metrics.classification_report(Y_test, Y_pred, digits=3))  # szczegolowa analiza, wszystkie miary


ValueError: could not convert string to float: "@VirginAmerica plus you've added commercials to the experience... tacky."

In [112]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', RandomForestClassifier(n_estimators=100, max_depth=None,min_samples_split=4, random_state=0)),
                      
])

text_clf.fit(X_train, Y_train) # trenowanie modelu
Y_pred = text_clf.predict(X_test) # przewidywanie etykiet danych testowych (Y_pred)


print(metrics.classification_report(Y_test, Y_pred, digits=3))  # szczegolowa analiza, wszystkie miary



              precision    recall  f1-score   support

    negative      0.920     0.987     0.952      2163
    positive      0.873     0.508     0.642       378

   micro avg      0.916     0.916     0.916      2541
   macro avg      0.896     0.747     0.797      2541
weighted avg      0.913     0.916     0.906      2541



In [27]:
from sklearn import svm
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', svm.SVC()),
 ])

text_clf.fit(X_train, Y_train) # trenowanie modelu
Y_pred = text_clf.predict(X_test) # przewidywanie etykiet danych testowych (Y_pred)


print(metrics.classification_report(Y_test, Y_pred))  # szczegolowa analiza, wszystkie miary


/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


              precision    recall  f1-score   support

    negative       0.85      1.00      0.92      2163
    positive       0.00      0.00      0.00       378

   micro avg       0.85      0.85      0.85      2541
   macro avg       0.43      0.50      0.46      2541
weighted avg       0.72      0.85      0.78      2541



/opt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [65]:
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', svm.SVC(C=5.0, cache_size=2000, class_weight=None, coef0=0.0, decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf', max_iter=-1, probability=False, random_state=None, shrinking=True, tol=0.001, verbose=False)),
 ])

text_clf.fit(X_train, Y_train) # trenowanie modelu
Y_pred = text_clf.predict(X_test) # przewidywanie etykiet danych testowych (Y_pred)


print(metrics.classification_report(Y_test, Y_pred))  # szczegolowa analiza, wszystkie miary



              precision    recall  f1-score   support

    negative       0.85      1.00      0.92      2163
    positive       0.00      0.00      0.00       378

   micro avg       0.85      0.85      0.85      2541
   macro avg       0.43      0.50      0.46      2541
weighted avg       0.72      0.85      0.78      2541



/opt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
